In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
plt.rc('figure', figsize=(20, 8.0))
#trying yahoo API dataset of Apple

amazon = pd.read_csv('./company_stock_prices/AMZN.csv')

In [3]:
def splits(data,train_ratio):
  train_length = int(len(data) * train_ratio)
  train_data = data[0:train_length]
  test_data = data[train_length+1:len(data)]
  return train_data,test_data

In [4]:
train_data,test_data=splits(amazon,0.7)
test_data.reset_index(inplace = True, drop = True)
n_diff = 2
train_x_diff = train_data['Close'] - train_data['Close'].shift(n_diff)
train_x_diff = train_x_diff[n_diff: ]
test_x_diff = test_data['Close'] - test_data['Close'].shift(n_diff)
test_x_diff = test_x_diff[n_diff: ]

trainDataArr = train_x_diff.values
testDataArr = test_x_diff.values
from statsmodels.tsa.arima_model import ARIMA
history = [x for x in trainDataArr]
# history
print(type(history))
predictions = list()
for i in range(len(testDataArr)):
    model = ARIMA(history, order=(5,0,1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    op = output[0]
    predictions.append(op)
    new = testDataArr[i]
    history.append(new)

actualValues = test_data
predictedValues = pd.DataFrame(predictions,columns=['PredictedDiff'])
actualValues['PreviousClose']=actualValues['Close'].shift(n_diff)
actualValues=actualValues[n_diff:]
actualValues.reset_index(inplace = True, drop = True)
actualValues['PredictedDiff']=predictedValues['PredictedDiff']
actualValues['PredictedClosePrice']=actualValues['PreviousClose']+actualValues['PredictedDiff']
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

error = mean_squared_error(actualValues['Close'], actualValues['PredictedClosePrice'])
print('Testing Mean Squared Error: %.3f' % error)

error = mean_squared_error(actualValues['Close'], actualValues['PredictedClosePrice'],squared=False)
print('Testing Root Mean Squared Error: %.3f' % error)

error = mean_absolute_error(actualValues['Close'], actualValues['PredictedClosePrice'])
print('Testing Mean absolute Error: %.3f' % error)
actualValues
actualValues[['Close', 'PredictedClosePrice']].plot(marker='.',color=['green', 'brown'])

<class 'list'>
